### imports 

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# import seaborn as sns
import math
# %matplotlib inline
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import category_encoders as ce
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import tree
import random
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import (RandomOverSampler, 
                                    SMOTE, 
                                    ADASYN)
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score
import keras
from keras.models import Sequential
from keras.layers import Dense
import datetime

Using TensorFlow backend.
C:\Users\nishant\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\nishant\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\nishant\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\nishant\Anaconda3\lib\site-packages\tensorflow\python\framework\d

### read data 

In [2]:
lo={'recommendation_set_id':str, 'user_id':str, 'session_id':str, 'query_identifier':str,
'query_word_count':float, 'query_char_count':float, 'query_detected_language':str,
'query_document_id':str, 'document_language_provided':str, 'year_published':float,
'number_of_authors':float, 'abstract_word_count':float, 'abstract_char_count':float,
'abstract_detected_language':str, 'first_author_id':str,
'num_pubs_by_first_author':float, 'organization_id':str, 'application_type':str,
'item_type':str, 'request_received':str, 'hour_request_received':str,
'response_delivered':str, 'rec_processing_time':float, 'app_version':str, 'app_lang':str,
'user_os':str, 'user_os_version':str, 'user_java_version':str, 'user_timezone':str,
'country_by_ip':str, 'timezone_by_ip':str, 'local_time_of_request':str,
'local_hour_of_request':str, 'number_of_recs_in_set':float,
'recommendation_algorithm_id_used':str, 'algorithm_class':str, 'cbf_parser':str,
'search_title':str, 'search_keywords':str, 'search_abstract':str,
'time_recs_recieved':str, 'time_recs_displayed':str, 'time_recs_viewed':str,
'clicks':float, 'ctr':float,'set_clicked':float
   }
# pars=['request_received', 'response_delivered','local_time_of_request','time_recs_recieved','time_recs_displayed','time_recs_viewed']
pars=['request_received']

df_w=pd.read_csv('tcdml1920-rec-click-pred--training-myvolt.csv',na_values=["\\N","nA"], dtype=lo, parse_dates=pars)
df_kag=pd.read_csv('tcdml1920-rec-click-pred--test.csv',na_values=["\\N","nA"], dtype=lo, parse_dates=pars)

### cleaning jabref 

In [3]:
# global dict_key
# dict_key={}
def process(df,run,dict_key):
#     global dict_key
#     dict_key={}
    
    df=df[df.organization_id=='4']
    df=df[[
        'user_id',
#         'session_id',
        'query_word_count', 'query_detected_language',
       'query_document_id','abstract_word_count','abstract_detected_language', 
       'item_type'
#            ,'request_received'
           ,'hour_request_received','country_by_ip', 'timezone_by_ip', 'local_hour_of_request',
       'recommendation_algorithm_id_used','set_clicked']]

# user_id
#     df['user_id_present']=df.user_id.isna()*1
#     df.drop(columns='user_id',inplace=True)
    
#     session_id
#     df['session_id']=df.session_id.isna()*1
#     df.drop(columns='session_id',inplace=True)
    
    # query_word_count
    
    # query_detected_language

    # query_document_id
#     df['query_doc_id_present']=df.query_document_id.isna()*1
#     df.drop(columns='query_document_id',inplace=True)

#     df['request_received']=df['request_received'].astype('datetime64[D]')
#     df['day_of_week'] = df['request_received'].dt.day_name()
#     df['month'] = df['request_received'].dt.month
#     df['year']=df['request_received'].dt.year
#     df['month']=df.month.astype(str)
#     df.drop(columns=['request_received'],inplace=True)

    # country_by_ip
#     df['country_by_ip'].fillna(df['country_by_ip'].mode()[0], inplace=True) #do after 'other' thingy 

#     timezone_by_ip
    # local_hour_of_request

    # recommendation_algorithm_id_used

    def convert_sparse_values(df, cols, threshold, replacement='other'):
        for col in [cols]:
            counts = df[col].value_counts()
            to_convert = counts[counts <= threshold].index.values
            dict_key[cols]=to_convert
            df[col] = df[col].replace(to_convert, replacement)

                
#     if run=='train':
#         convert_sparse_values(df,cols='query_detected_language', threshold=1000)
#         convert_sparse_values(df,cols='app_lang', threshold=500)
#         convert_sparse_values(df,cols='country_by_ip', threshold=150)
#         convert_sparse_values(df,cols='timezone_by_ip', threshold=500)
#         convert_sparse_values(df,cols='app_version', threshold=800)

#     else:
#         df['query_detected_language']=df.query_detected_language.map(lambda x: x if x in dict_key['query_detected_language'] else 'others')
#         df['app_lang']=df.app_lang.map(lambda x: x if x in dict_key['app_lang'] else 'others')
#         df['country_by_ip']=df.country_by_ip.map(lambda x: x if x in dict_key['country_by_ip'] else 'others')
#         df['timezone_by_ip']=df.timezone_by_ip.map(lambda x: x if x in dict_key['country_by_ip'] else 'others')
#         df['app_version']=df.app_version.map(lambda x: x if x in dict_key['app_version'] else 'others')
        
#     df['country_by_ip'].fillna(df['country_by_ip'].mode()[0], inplace=True)
#     df.timezone_by_ip.fillna(df.timezone_by_ip.mode()[0],inplace=True) 

    df.drop(columns=['abstract_word_count'],inplace=True)
    
    df=df.drop(columns=['set_clicked']).merge(df[['set_clicked']], 
                                    on=df[['set_clicked']].index).drop(columns='key_0')
    
    return df,dict_key


df_w_p,d=process(df_w,run='train',dict_key={}) #train

df_kag_p,d=process(df_kag,run='test',dict_key=d) #kaggle


In [4]:
kags=[]
# for i in range(10):
if 1==0:
    
    print("---------------------------")
    sig=random.randrange(1,20)/10
#     print("sig",sig)
    encoder=ce.LeaveOneOutEncoder()
    encoder.fit(X=df_w_p.iloc[:,:-1],y=df_w_p['set_clicked'])
    df_w_encoded=encoder.transform(df_w_p.iloc[:,:-1])
    df_kag_encoded=encoder.transform(df_kag_p.iloc[:,:-1])

    df_w_encoded['set_clicked']=df_w_p.iloc[:,-1]
    df_kag_encoded['set_clicked']=df_kag_p.iloc[:,-1]

    sig=random.randrange(3,20)
#     print("neighbors",sig)
    est=IterativeImputer(random_state=0, estimator= KNeighborsRegressor(n_neighbors=30))
    est.fit(df_w_encoded)
    X_w_im=est.transform(df_w_encoded)
    X_kag_im=est.transform(df_kag_encoded)

    X_w=X_w_im[:,:-1]
    X_kag=X_kag_im[:,:-1]
    y_w=df_w_p.iloc[:,-1]

    sig=random.randrange(5,40)/100
    print("split",sig)
    x_train, x_test, y_train, y_test = train_test_split(X_w, y_w, test_size=sig, stratify=y_w)

    sampler = SMOTE(ratio=0.2)
#     x_train, y_train = sampler.fit_sample(x_train, y_train)

    sig=random.randrange(3,8)
    print("depth",sig)
    clf = tree.DecisionTreeClassifier(max_depth=sig)
    clf.fit(x_train,y_train)
    preds_class = clf.predict(x_test)
    acc=sum(y_test==preds_class)/len(y_test)
#     print("    accuracy: ",acc)
#     print("    f1: ",f1_score(y_test, preds_class))
    print("    pr: ",precision_score(y_test, preds_class))
#     print("    cm:",confusion_matrix(y_test, preds_class))

    print("    kaggle: ",sum(clf.predict(X_kag)))
    if precision_score(y_test, preds_class)>.57:
        print("yo")
        kags.append(clf.predict(X_kag))

In [5]:
# pd.DataFrame(kags).T.to_csv('aa.csv')

### category encodering 

In [6]:
def encode_all(df,df2,encoder_to_use,handle_missing='return_nan'):  #handle_missing='value'/'return_nan'
    encoders_used={}
    for col in encoder_to_use:
        encoders_used[col]={}

        if encoder_to_use[col]=='BinaryEncoder':
            encoder=ce.BinaryEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder'
            encoders_used[col]['encoder_object']=encoder

        if encoder_to_use[col]=='CatBoostEncoder':
            encoder=ce.CatBoostEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder'
            encoders_used[col]['encoder_object']=encoder

        if encoder_to_use[col]=='LeaveOneOutEncoder':
            encoder=ce.LeaveOneOutEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder' 
            encoders_used[col]['encoder_object']=encoder

        if encoder_to_use[col]=='OneHotEncoder':
            encoder=ce.OneHotEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing,use_cat_names=True)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder'
            encoders_used[col]['encoder_object']=encoder

        if encoder_to_use[col]=='OrdinalEncoder':
            encoder=ce.OrdinalEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder' 
            encoders_used[col]['encoder_object']=encoder

        if encoder_to_use[col]=='TargetEncoder':
            encoder=ce.TargetEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder' 
            encoders_used[col]['encoder_object']=encoder


        if encoder_to_use[col]=='WOEEncoder':
            encoder=ce.WOEEncoder(cols=[col],return_df=1,drop_invariant=1,handle_missing=handle_missing)
            encoder.fit(X=df,y=df['set_clicked'])
            df=encoder.transform(df)
            df2=encoder.transform(df2)
            encoders_used[col]['encoder_type']='BackwardDifferenceEncoder' 
            encoders_used[col]['encoder_object']=encoder
            
            
    return df,df2

### random encoders and imputers

In [ ]:


encoders_to_use=[
{
'query_detected_language':{'name':'TargetEncoder','min_samples_leaf':10, 'smoothing':4.0},
'first_author_id':{'name':'TargetEncoder','min_samples_leaf':10, 'smoothing':4.0},
'hour_request_received':{'name':'BaseNEncoder','base':2},
'app_version':{'name':'TargetEncoder','min_samples_leaf':10, 'smoothing':4.0},
'app_lang':{'name':'TargetEncoder','min_samples_leaf':10, 'smoothing':4.0},
'country_by_ip':{'name':'TargetEncoder','min_samples_leaf':10, 'smoothing':4.0},
'timezone_by_ip':{'name':'TargetEncoder','min_samples_leaf':10, 'smoothing':4.0},
'local_hour_of_request':{'name':'BaseNEncoder','base':2},
'recommendation_algorithm_id_used':{'name':'BaseNEncoder','base':2},
'day_of_week':{'name':'BaseNEncoder','base':2},
'month':{'name':'BaseNEncoder','base':2}
},
    {
'query_detected_language':{'name':'MEstimateEncoder','randomized':True, 'sigma':0.1,'m':2},
'first_author_id':{'name':'MEstimateEncoder','randomized':True, 'sigma':0.1,'m':2},
'hour_request_received':{'name':'BaseNEncoder','base':2},
'app_version':{'name':'MEstimateEncoder','randomized':True, 'sigma':0.1,'m':2},
'app_lang':{'name':'MEstimateEncoder','randomized':True, 'sigma':0.1,'m':2},
'country_by_ip':{'name':'MEstimateEncoder','randomized':True, 'sigma':0.1,'m':2},
'timezone_by_ip':{'name':'MEstimateEncoder','randomized':True, 'sigma':0.1,'m':2},
'local_hour_of_request':{'name':'BaseNEncoder','base':2},
'recommendation_algorithm_id_used':{'name':'BaseNEncoder','base':2},
'day_of_week':{'name':'BaseNEncoder','base':2},
'month':{'name':'BaseNEncoder','base':2}
},
    {
'query_detected_language':{'name':'LeaveOneOutEncoder','sigma':.3},
'first_author_id':{'name':'LeaveOneOutEncoder','sigma':.3},
'hour_request_received':{'name':'BaseNEncoder','base':2},
'app_version':{'name':'LeaveOneOutEncoder','sigma':.3},
'app_lang':{'name':'LeaveOneOutEncoder','sigma':.3},
'country_by_ip':{'name':'LeaveOneOutEncoder','sigma':.3},
'timezone_by_ip':{'name':'LeaveOneOutEncoder','sigma':.3},
'local_hour_of_request':{'name':'BaseNEncoder','base':2},
'recommendation_algorithm_id_used':{'name':'BaseNEncoder','base':2},
'day_of_week':{'name':'BaseNEncoder','base':2},
'month':{'name':'BaseNEncoder','base':2}
},
    {
'query_detected_language':{'name':'LeaveOneOutEncoder','sigma':.1},
'first_author_id':{'name':'LeaveOneOutEncoder','sigma':.1},
'hour_request_received':{'name':'BaseNEncoder','base':2},
'app_version':{'name':'LeaveOneOutEncoder','sigma':.1},
'app_lang':{'name':'LeaveOneOutEncoder','sigma':.1},
'country_by_ip':{'name':'LeaveOneOutEncoder','sigma':.1},
'timezone_by_ip':{'name':'LeaveOneOutEncoder','sigma':.1},
'local_hour_of_request':{'name':'BaseNEncoder','base':2},
'recommendation_algorithm_id_used':{'name':'BaseNEncoder','base':2},
'day_of_week':{'name':'BaseNEncoder','base':2},
'month':{'name':'BaseNEncoder','base':2}
},
    {
'query_detected_language':{'name':'WOEEncoder','randomized':True, 'sigma':0.1},
'first_author_id':{'name':'WOEEncoder','randomized':True, 'sigma':0.1},
'hour_request_received':{'name':'BaseNEncoder','base':2},
'app_version':{'name':'WOEEncoder','randomized':True, 'sigma':0.1},
'app_lang':{'name':'WOEEncoder','randomized':True, 'sigma':0.1},
'country_by_ip':{'name':'WOEEncoder','randomized':True, 'sigma':0.1},
'timezone_by_ip':{'name':'WOEEncoder','randomized':True, 'sigma':0.1},
'local_hour_of_request':{'name':'BaseNEncoder','base':2},
'recommendation_algorithm_id_used':{'name':'BaseNEncoder','base':2},
'day_of_week':{'name':'BaseNEncoder','base':2},
'month':{'name':'BaseNEncoder','base':2}
},
    {
'query_detected_language':{'name':'WOEEncoder','randomized':True, 'sigma':0.3},
'first_author_id':{'name':'WOEEncoder','randomized':True, 'sigma':0.3},
'hour_request_received':{'name':'BaseNEncoder','base':2},
'app_version':{'name':'WOEEncoder','randomized':True, 'sigma':0.3},
'app_lang':{'name':'WOEEncoder','randomized':True, 'sigma':0.3},
'country_by_ip':{'name':'WOEEncoder','randomized':True, 'sigma':0.3},
'timezone_by_ip':{'name':'WOEEncoder','randomized':True, 'sigma':0.3},
'local_hour_of_request':{'name':'BaseNEncoder','base':2},
'recommendation_algorithm_id_used':{'name':'BaseNEncoder','base':2},
'day_of_week':{'name':'BaseNEncoder','base':2},
'month':{'name':'BaseNEncoder','base':2}
}
]
    
encoders_used={}

def make_encoder_combinations(df):
    return random.choice(encoders_to_use)

### all 

In [ ]:

kag_pred_outer=[]

enc=make_encoder_combinations(df_w_p)
for xx,encoder_to_use in enumerate(enc):
    
    print("----------------------------------------------")
    print("encoding: ",xx," of ",len(enc))
    df_w_encoded, df_kag_encoded=encode_all(df_w_p.copy(), df_kag_p.copy(),encoder_to_use)

    print("imputing...")

#     random.make_imputer_combinations(df_w_p)
    
    est=IterativeImputer(random_state=0, estimator= KNeighborsRegressor())
    est.fit(df_w_encoded)
    X_w_im=est.transform(df_w_encoded)
    X_kag_im=est.transform(df_kag_encoded)

    X_w=X_w_im[:,:-1]
    X_kag=X_kag_im[:,:-1]
    y_w=df_w_p.iloc[:,-1]

    x=random.randrange(15,40)/100
    print("train-test-split: ",x)
    x_train, x_test, y_train, y_test = train_test_split(X_w, y_w, test_size=x, stratify=y_w)

#     all models

    test_preds=[]
    kag_preds=[]

    print("training models...")
    

    model1 = tree.DecisionTreeClassifier(max_depth=4)
    model1.fit(x_train,y_train)
    test_pred1=model1.predict(x_test)
    kag_pred1=model1.predict(X_kag)
    test_preds.append(test_pred1)
    kag_preds.append(kag_pred1)
    print("    sum_of_kag1 DT",kag_pred1.sum())

    model2 = CatBoostClassifier(iterations=1500,learning_rate=.02,depth=4,task_type="GPU",devices='0:2')
    model2.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=False)
    test_pred2=model2.predict(x_test)
    kag_pred2=model2.predict(X_kag)
    test_preds.append(test_pred2)
    kag_preds.append(kag_pred2)
    print("    sum_of_kag2 Cat",kag_pred2.sum())

    model3 = RandomForestClassifier(n_estimators=800)
    model3.fit(x_train,y_train)
    test_pred3=model3.predict(x_test)
    kag_pred3=model3.predict(X_kag)
    test_preds.append(test_pred3)
    kag_preds.append(kag_pred3)
    print("    sum_of_kag3 RF",kag_pred3.sum())

    model4 = tree.DecisionTreeClassifier(max_depth=5)
    model4.fit(x_train,y_train)
    test_pred4=model4.predict(x_test)
    kag_pred4=model4.predict(X_kag)
    test_preds.append(test_pred4)
    kag_preds.append(kag_pred4)
    print("    sum_of_kag4 DT",kag_pred4.sum())


    # Neural network
    model5 = Sequential()
    model5.add(Dense(5, input_dim=x_train.shape[1], activation='relu'))
    # model.add(Dense(12, activation='relu'))
    model5.add(Dense(1, activation='sigmoid'))
    model5.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model5.fit(x_train, y_train,validation_data = (x_test,y_test), epochs=10, batch_size=10, verbose=0)
    test_pred5=model5.predict(x_test)
    kag_pred5=model5.predict(X_kag)
    test_preds.append(test_pred5)
    kag_preds.append(kag_pred5)
    print("    sum_of_kag5 NN",kag_pred5.sum())

    model6 = LogisticRegression(random_state=0, solver='lbfgs',max_iter=1000).fit(x_train, y_train)
    model6.fit(x_train,y_train)
    test_pred6=model6.predict(x_test)
    kag_pred6=model6.predict(X_kag)
    test_preds.append(test_pred6)
    kag_preds.append(kag_pred6)
    print("    sum_of_kag6 LR",kag_pred6.sum())

    model7 = xgb.XGBClassifier(depth=3,learning_rate=0.2,iterations=1000)
    model7.fit(x_train,y_train)
    test_pred7=model7.predict(x_test)
    kag_pred7=model4.predict(X_kag)
    test_preds.append(test_pred7)
    kag_preds.append(kag_pred7)
    print("    sum_of_kag7 XGB",kag_pred7.sum())

    model8 = xgb.XGBClassifier(depth=4,learning_rate=0.2,iterations=1000)
    model8.fit(x_train,y_train)
    test_pred8=model8.predict(x_test)
    kag_pred8=model8.predict(X_kag)
    test_preds.append(test_pred8)
    kag_preds.append(kag_pred8)
    print("    sum_of_kag8 XGB",kag_pred8.sum())



    model9 = xgb.XGBClassifier(depth=5,learning_rate=0.2,iterations=1000)
    model9.fit(x_train,y_train)
    test_pred9=model9.predict(x_test)
    kag_pred9=model9.predict(X_kag)
    test_preds.append(test_pred9)
    kag_preds.append(kag_pred9)
    print("    sum_of_kag9 XGB",kag_pred9.sum())


    model10 = CatBoostClassifier(iterations=1500,learning_rate=.02,depth=5,task_type="GPU",devices='0:2')
    model10.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=False)
    test_pred10=model10.predict(x_test)
    kag_pred10=model10.predict(X_kag)
    test_preds.append(test_pred10)
    kag_preds.append(kag_pred10)
    print("    sum_of_kag10 CAT",kag_pred10.sum())

    #oversampling
    sampler = SMOTE(ratio=0.3)
    x_train, y_train = sampler.fit_sample(x_train, y_train)

    print("training models after oversampling...")
    model11 = tree.DecisionTreeClassifier(max_depth=4)
    model11.fit(x_train,y_train)
    test_pred11=model11.predict(x_test)
    kag_pred11=model1.predict(X_kag)
    test_preds.append(test_pred11)
    kag_preds.append(kag_pred11)
    print("    sum_of_kag11 DT",kag_pred11.sum())

    model12 = CatBoostClassifier(iterations=1500,learning_rate=.02,depth=4,task_type="GPU",devices='0:2')
    model12.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=False)
    test_pred12=model12.predict(x_test)
    kag_pred12=model12.predict(X_kag)
    test_preds.append(test_pred12)
    kag_preds.append(kag_pred12)
    print("    sum_of_kag12 CAT",kag_pred12.sum())

    model13 = RandomForestClassifier(n_estimators=400)
    model13.fit(x_train,y_train)
    test_pred13=model13.predict(x_test)
    kag_pred13=model13.predict(X_kag)
    test_preds.append(test_pred13)
    kag_preds.append(kag_pred13)
    print("    sum_of_kag13 RF",kag_pred13.sum())

    model14 = tree.DecisionTreeClassifier(max_depth=5)
    model14.fit(x_train,y_train)
    test_pred14=model14.predict(x_test)
    kag_pred14=model14.predict(X_kag)
    test_preds.append(test_pred14)
    kag_preds.append(kag_pred14)
    print("    sum_of_kag14 DT",kag_pred14.sum())

    # Neural network
    model15 = Sequential()
    model15.add(Dense(5, input_dim=x_train.shape[1], activation='relu'))
    # model1.add(Dense(12, activation='relu'))
    model15.add(Dense(1, activation='sigmoid'))
    model15.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model15.fit(x_train, y_train,validation_data = (x_test,y_test), epochs=10, batch_size=10, verbose=0)
    test_pred15=model15.predict(x_test)
    kag_pred15=model15.predict(X_kag)
    test_preds.append(test_pred15)
    kag_preds.append(kag_pred15)
    print("    sum_of_kag15 NN",kag_pred15.sum())

    model16 = LogisticRegression(random_state=0, solver='lbfgs',max_iter=1000).fit(x_train, y_train)
    model16.fit(x_train,y_train)
    test_pred16=model16.predict(x_test)
    kag_pred16=model16.predict(X_kag)
    test_preds.append(test_pred16)
    kag_preds.append(kag_pred16)
    print("    sum_of_kag16 LR",kag_pred16.sum())

    model17 = xgb.XGBClassifier(depth=3,learning_rate=0.2,iterations=1000)
    model17.fit(x_train,y_train)
    test_pred17=model17.predict(x_test)
    kag_pred17=model14.predict(X_kag)
    test_preds.append(test_pred17)
    kag_preds.append(kag_pred17)
    print("    sum_of_kag17 XGB",kag_pred17.sum())

    model18 = xgb.XGBClassifier(depth=4,learning_rate=0.2,iterations=1000)
    model18.fit(x_train,y_train)
    test_pred18=model18.predict(x_test)
    kag_pred18=model18.predict(X_kag)
    test_preds.append(test_pred18)
    kag_preds.append(kag_pred18)
    print("    sum_of_kag18 XGB",kag_pred18.sum())


    model19 = xgb.XGBClassifier(depth=5,learning_rate=0.2,iterations=1000)
    model19.fit(x_train,y_train)
    test_pred19=model19.predict(x_test)
    kag_pred19=model19.predict(X_kag)
    test_preds.append(test_pred19)
    kag_preds.append(kag_pred19)
    print("    sum_of_kag19 XGB",kag_pred19.sum())

    model20 = CatBoostClassifier(iterations=1500,learning_rate=.02,depth=5,task_type="GPU",devices='0:2')
    model20.fit(x_train,y_train, eval_set=(x_test,y_test),verbose=False)
    test_pred20=model20.predict(x_test)
    kag_pred20=model20.predict(X_kag)
    test_preds.append(test_pred20)
    kag_preds.append(kag_pred20)
    print("    sum_of_kag20 CAT",kag_pred20.sum())


    # stack
    print("stacking...")
    test_df=pd.DataFrame(test_preds).T
    kag_df=pd.DataFrame(kag_preds).T

    model = LogisticRegression(solver='liblinear')
    model.fit(test_df,y_test)
    kag_preds=model.predict(kag_df)
    print("SUM_of_kag",kag_preds.sum())
    
    kag_pred_outer.append(kag_preds)

    
pd.DataFrame(kag_pred_outer).T.to_csv('aa.csv')

----------------------------------------------
encoding:  0  of  29
imputing...


C:\Users\nishant\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


train-test-split:  0.29
training models...
    sum_of_kag1 DT 6.0
    sum_of_kag2 Cat 6.0
    sum_of_kag3 RF 18.0
    sum_of_kag4 DT 5.0
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
    sum_of_kag5 NN 41.632584
    sum_of_kag6 LR 4.0
    sum_of_kag7 XGB 5.0
    sum_of_kag8 XGB 2.0
    sum_of_kag9 XGB 2.0


    sum_of_kag10 CAT 9.0
training models after oversampling...
    sum_of_kag11 DT 6.0


    sum_of_kag12 CAT 239.0
    sum_of_kag13 RF 138.0
    sum_of_kag14 DT 309.0
    sum_of_kag15 NN 641.11096
    sum_of_kag16 LR 73.0
    sum_of_kag17 XGB 309.0
    sum_of_kag18 XGB 257.0
    sum_of_kag19 XGB 257.0


...

C:\Users\nishant\Anaconda3\lib\site-packages\sklearn\impute\_iterative.py:603: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  " reached.", ConvergenceWarning)


train-test-split:  0.19
training models...
    sum_of_kag1 DT 2.0


    sum_of_kag2 Cat 7.0
    sum_of_kag3 RF 13.0
    sum_of_kag4 DT 4.0
    sum_of_kag5 NN 35.484398
    sum_of_kag6 LR 5.0
    sum_of_kag7 XGB 4.0
    sum_of_kag8 XGB 3.0
    sum_of_kag9 XGB 3.0


    sum_of_kag10 CAT 7.0
training models after oversampling...
    sum_of_kag11 DT 2.0


    sum_of_kag12 CAT 145.0
    sum_of_kag13 RF 56.0
    sum_of_kag14 DT 322.0


In [ ]:
x_train.shape